# iFood Idea Pitch

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats.mstats import winsorize
from math import ceil
from datetime import timedelta
import os

import warnings
warnings.filterwarnings('ignore')

from matplotlib.colors import LinearSegmentedColormap

import json
from ast import literal_eval


## Data Exploration

In [2]:
data = pd.read_csv("recipes_w_search_terms.csv", 
                   converters={'ingredients': literal_eval, 
                               'tags':literal_eval
                              },
                   nrows=400
                  )


In [3]:
data.head(3)

,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms
0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to ser...","[water, grits, salt, cheddar cheese, garlic, o...","[""4 cups water"",""1 cup uncooked old f...",1 (155 g),8,"['I a sauce pan, bring water to a boil; slowly...","[time-to-make, course, main-ingredient, prepar...","{'diabetic', 'low-calorie', 'vegetarian', 'low..."
1,232037,Simple Shrimp and Andouille Jambalaya,"Simple, easy and very tasty for when you are i...","[onion, red bell pepper, garlic cloves, large ...","[""1 medium onion, chopped coarse "",""1 m...",1 (366 g),4,"['In a food processor, pulse the onion, red pe...","[60-minutes-or-less, time-to-make, course, mai...","{'dinner', 'shrimp'}"
2,41090,black-and-white bean salad,NaN,"[white beans, canned black beans, tomatoes, on...","[""1 cup canned white beans, rinsed and dra...",1 (807 g),1,"['In a large bowl, combine beans, tomato, onio...","[15-minutes-or-less, time-to-make, course, mai...","{'vegetarian', 'salad', 'side', 'dinner', 'veg..."


In [4]:
data['search_terms'] = data['search_terms'].apply(lambda x:x[1:-1].split(',')).apply(lambda x:[str(i) for i in x])

In [5]:
ingredients_index = pd.DataFrame(columns=['ingredients'])
ingredients_index = ingredients_index.append(pd.DataFrame(data['ingredients'],columns=['ingredients']), ignore_index=True)
ingredients_index['recipe_id'] = data['id']
ingredients_index['recipe_name'] = data['name']

ingredients_index = ingredients_index.explode('ingredients').reset_index(drop=True)


In [6]:
unique_ingredients = pd.DataFrame(ingredients_index['ingredients'].unique().tolist(), 
                                  columns=['ingredients'])

In [7]:
tags_index = pd.DataFrame(columns=['tags'])
tags_index = tags_index.append(pd.DataFrame(data['tags'],columns=['tags']), ignore_index=True)
tags_index['recipe_id'] = data['id']
tags_index['recipe_name'] = data['name']

tags_index = tags_index.explode('tags').reset_index(drop=True)


In [8]:
unique_tags = pd.DataFrame(tags_index['tags'].unique().tolist(),
                           columns=['tags'])

In [9]:
terms_index = pd.DataFrame(columns=['search_terms'])
terms_index = terms_index.append(pd.DataFrame(data['search_terms'],columns=['search_terms']), ignore_index=True)
terms_index['recipe_id'] = data['id']
terms_index = terms_index.explode('search_terms').reset_index(drop=True)


In [10]:
unique_terms = pd.DataFrame(terms_index['search_terms'].unique().tolist(),
                            columns=['search_terms'])

In [11]:
data_cols = ['id', 'name', 'description', 'ingredients_raw_str',
       'serving_size', 'servings', 'steps',]

data = data[data_cols]

In [12]:
ingredients_index.loc[ingredients_index['ingredients']=='red wine',:]

,ingredients,recipe_id,recipe_name
2010,red wine,360683,Spaghetti a La Melissa
3594,red wine,329598,Crispy Skinned Duck With Truffled Red Currant ...


In [14]:
tags_index.loc[tags_index['tags']=='seafood',:]

,tags,recipe_id,recipe_name
33,seafood,232037,Simple Shrimp and Andouille Jambalaya
146,seafood,79222,Potato-Crab Chowder
282,seafood,232099,Scarlett's Crock Pot Cheese (And Prawn) Fondue...
525,seafood,232108,Shrimp Stir-Fry
635,seafood,350688,Roasted Shrimp Cocktail
715,seafood,127155,Crab &amp; Fresh Basil Stuffed Mushrooms
820,seafood,232073,Kickin' Shrimp Dip
1374,seafood,161899,Crock Pot Jambalaya
1776,seafood,56494,Sweet Chilli Prawn Cakes
2456,seafood,281620,Salmon Patties
